In [57]:
import pandas as pd

df = pd.read_csv("../data/final/merged_rewe_fdc_insulin_time.csv")

# df = pd.read_csv("../data/raw/kaggle_satiety_dataset/satietyindex.csv")
# df.sort_values(by="Insulin Index")
# df.rename(columns=
#           {
#             "Insulin Index": "'Non Nutrient Data.Insulin Index", 
#             "Insulin Index Food Name": "Non Nutrient Data.Insulin Index Food Name",
#             "'Non Nutrient Data.Insulin Index": "Non Nutrient Data.Insulin Index",
#             }, 
# inplace=True)

# df.to_csv("../data/final/merged_rewe_fdc_insulin_time.csv", index=False)



# df['Non Nutrient Data.Preparation Time'].fillna(df['Non Nutrient Data.Preparation Time'].mean(), inplace=True)
# df.to_csv("../data/final/merged_rewe_fdc_insulin_time.csv", index=False)

In [25]:
# insulin_path_1 = "../data/raw/insulin_index_food_list_1.csv"
# insulin_df_1 = pd.read_csv(insulin_path_1)

# insulin_path_2 = "../data/raw/insulin_index_food_list_2.csv"
# insulin_df_2 = pd.read_csv(insulin_path_2)

# insulin_df_1.drop(columns=["Weight (g/MJ)", "Food Test"], inplace=True)
# insulin_df_1.rename(columns={"FII (%)": "Insulin Index"}, inplace=True)

# insulin_df_2.rename(columns={"Food" : "Name"}, inplace=True)

# insulin_df = pd.concat([insulin_df_1, insulin_df_2], axis=0).sort_values(by="Insulin Index")

# insulin_df = insulin_df[~insulin_df['Name'].duplicated(keep=False)].reset_index(drop=True)
# insulin_df["Name"] = insulin_df["Name"].str.replace('"', '', regex=False)
# insulin_df.rename(columns={"Name": "Insulin Index Food Name"}, inplace=True)
# insulin_df.to_csv("../data/processed/insulin_index.csv", index=False)


In [54]:
import pandas as pd
import requests
from io import BytesIO
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.formatting.rule import ColorScaleRule
from openpyxl.formatting import Rule
from openpyxl.styles.differential import DifferentialStyle
from datetime import datetime, timedelta

def create_weekly_sheets(wb, df, start_date, shopping_day):
    current_date = start_date
    max_weeks = 2#df['Shopping Period (weeks)'].max()
    header_font = Font(bold=True, color="FFFFFF")
    header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
    even_fill = PatternFill(start_color="F5F5F5", end_color="F5F5F5", fill_type="solid")

    for week in range(1, max_weeks + 1):
        ws_week = wb.create_sheet(title=f"Week {week} ({(current_date + timedelta(weeks=week - 1)).strftime('%Y-%m-%d')})")
        shopping_date = current_date + timedelta(weeks=week - 1)
        
        # Filter items for the current week, include all items for week 1
        if week == 1:
            week_df = df.copy()
        else:
            week_df = df[(df['Shopping Period (weeks)'] == 1) | ((week - 1) % df['Shopping Period (weeks)'] == 0)].copy()
        
        # Remove the URL column for weekly sheets
        week_df_no_url = week_df.drop(columns=['Non Nutrient Data.Image URL'])

        # Add shopping date at the top
        ws_week.append(["Shopping Date:", shopping_date.strftime('%Y-%m-%d')])
        ws_week.append([])

        # Add column headers with the "Status" column
        headers = ["Status"] + list(week_df_no_url.columns)
        ws_week.append(headers)
        
        # Style the header
        for cell in ws_week[3]:
            cell.font = header_font
            cell.fill = header_fill
        
        # Add DataFrame to the weekly sheet, shifting columns by one to the right
        for r in dataframe_to_rows(week_df_no_url, index=False, header=False):
            ws_week.append([""] + r)

        # Add images to the weekly sheet
        image_urls_week = week_df["Non Nutrient Data.Image URL"]
        image_row_week = 4  # Start from row 4 (row 1-3 contain headers)

        for image_url in image_urls_week:
            # Download the image
            response = requests.get(image_url)
            image_data = BytesIO(response.content)

            standard_row_height = 15
            ws_week.row_dimensions[image_row_week].height = standard_row_height * 2  # Set the height of the row to twice the standard height

            # Insert the image into the worksheet with offsets
            img = openpyxl.drawing.image.Image(image_data)
            img.width, img.height = img.width * 0.22, img.height * 0.22  # Scale the image
            ws_week.add_image(img, f'A{image_row_week}')

            # Move to the next row
            image_row_week += 1

        # Add data validation for "MISSING" and "GOT IT"
        dv_missing_gotit = DataValidation(type="list", formula1='"MISSING,GOT IT"', allow_blank=True)
        ws_week.add_data_validation(dv_missing_gotit)
        
        for row in range(4, len(week_df) + 4):
            cell_number = f'B{row}'
            cell = ws_week[cell_number]
            cell.value = "MISSING"
            dv_missing_gotit.add(cell)

        # Apply alternating row colors
        for row in ws_week.iter_rows(min_row=4, max_row=len(week_df) + 3, min_col=2, max_col=len(week_df_no_url.columns) + 1):
            for cell in row:
                if cell.row % 2 == 0:
                    cell.fill = even_fill

        # Apply dynamic conditional formatting for the "MISSING" and "GOT IT" statuses
        red_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
        green_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")

        red_rule = Rule(type="expression", dxf=DifferentialStyle(fill=red_fill))
        red_rule.formula = ['B4="MISSING"']

        green_rule = Rule(type="expression", dxf=DifferentialStyle(fill=green_fill))
        green_rule.formula = ['B4="GOT IT"']

        ws_week.conditional_formatting.add(f'B4:B{len(week_df) + 3}', red_rule)
        ws_week.conditional_formatting.add(f'B4:B{len(week_df) + 3}', green_rule)

        # Adjust column widths
        for col in ws_week.columns:
            max_length = 0
            column = col[0].column_letter  # Get the column name
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = max_length #+ 2
            #ws_week.column_dimensions[column].width = adjusted_width

# Read the CSV file into a DataFrame
df = pd.read_csv("../data/processed/merged_optimization_result.csv")
output_path = "../reports/shopping_list.xlsx"

df = df[['Non Nutrient Data.Image URL',
         'Shopping List.Product Name', 
         'Shopping List.Optimal Weekly Quantity (g)', 
         'Shopping List.Product Package Quantity Per Period (units / week)',
         'Shopping List.Product Package Weight (g)',
         'Shopping List.Shopping Period (weeks)',
         'Shopping List.Average Weekly Price (EUR)']]
df.columns = [col.replace('Shopping List.', '') for col in df.columns]

# Initialize a new workbook
wb = Workbook()
ws_overview = wb.active
ws_overview.title = "Overview"

# Create a DataFrame without the URL column for the sheet
df_no_url = df.drop(columns=['Non Nutrient Data.Image URL'])

# Add DataFrame to "Overview" sheet, shifting columns by one to the right
for r_idx, row in enumerate(dataframe_to_rows(df_no_url, index=False, header=True)):
    ws_overview.append([""] + row)

# Style the header
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")

for cell in ws_overview[1]:
    cell.font = header_font
    cell.fill = header_fill

# Insert images into column A of the "Overview" sheet
image_urls = df["Non Nutrient Data.Image URL"]
image_row = 2  # Start from row 2 (row 1 contains headers)

for image_url in image_urls:
    # Download the image
    response = requests.get(image_url)
    image_data = BytesIO(response.content)

    standard_row_height = 15
    ws_overview.row_dimensions[image_row].height = standard_row_height * 2  # Set the height of the row to twice the standard height

    # Insert the image into the worksheet with offsets
    img = openpyxl.drawing.image.Image(image_data)
    img.width, img.height = img.width * 0.22, img.height * 0.22  # Scale the image
    ws_overview.add_image(img, f'A{image_row}')

    # Move to the next row
    image_row += 1

# Add total average weekly price at the bottom
total_price = df['Average Weekly Price (EUR)'].sum()
ws_overview.append(["", "", "", "", "","Total", total_price])

# Style the total row
ws_overview[f'F{len(df) + 2}'].font = Font(bold=True)

# Adjust column widths
for col in ws_overview.columns:
    max_length = 0
    column = col[0].column_letter  # Get the column name
    for cell in col:
        try:
            if len(str(cell.value)) > max_length:
                max_length = len(cell.value)
        except:
            pass
    adjusted_width = max_length #+ 2
    #ws_overview.column_dimensions[column].width = adjusted_width

# Apply alternating row colors
even_fill = PatternFill(start_color="F5F5F5", end_color="F5F5F5", fill_type="solid")

for row in ws_overview.iter_rows(min_row=2, max_row=len(df) + 1, min_col=2, max_col=len(df_no_url.columns) + 1):
    for cell in row:
        if cell.row % 2 == 0:
            cell.fill = even_fill

# Calculate the starting row for new inputs based on the length of the DataFrame

# # Calculate the starting row for new inputs based on the length of the DataFrame
# start_row = len(df) + 4

# # Add user input fields
# ws_overview.append([""])
# ws_overview.append(["Enter your preferred shopping day:", ""])
# ws_overview.append(["Enter the current date (yyyy-mm-dd):", ""])

# # Add data validation for preferred shopping day
# dv_days = DataValidation(type="list", formula1='"Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday"', allow_blank=True)
# ws_overview.add_data_validation(dv_days)
# preferred_day_cell = f'B{start_row}'
# cell = ws_overview[preferred_day_cell]
# cell.value = "Monday"
# dv_days.add(cell)

# # Add data validation for current date input
# dv_date = DataValidation(type="date", formula1="DATE(2000,1,1)", formula2="DATE(2100,12,31)", allow_blank=True, showErrorMessage=True)
# dv_date.error = 'Enter a valid date in yyyy-mm-dd format'
# dv_date.errorTitle = 'Invalid Date'
# ws_overview.add_data_validation(dv_date)
# current_date_cell = f'B{start_row + 1}'
# cell = ws_overview[current_date_cell]
# cell.value = datetime.now().strftime("%Y-%m-%d")
# dv_date.add(cell)

# # Add explanatory text for date format
# ws_overview.cell(row=start_row + 1, column=3).value = "(format: yyyy-mm-dd)"

# # Add shopping status table
# ws_overview.append([""])
# ws_overview.append(["Shopping Date", "Status"])

# # Style the header
# header_font = Font(bold=True, color="FFFFFF")
# header_fill = PatternFill(start_color="000000", end_color="000000", fill_type="solid")
# header_row = start_row + 4
# for cell in ws_overview[header_row]:
#     cell.font = header_font
#     cell.fill = header_fill

# Placeholder for shopping dates calculation
# shopping_dates_start_row = header_row + 1

# # Add placeholder rows for shopping dates and status
# max_weeks = df['Shopping Period (weeks)'].max()
# for i in range(max_weeks):
#     ws_overview.append(["", "TODO"])

# # Apply alternating row colors
# even_fill = PatternFill(start_color="F5F5F5", end_color="F5F5F5", fill_type="solid")
# for row in ws_overview.iter_rows(min_row=shopping_dates_start_row, max_row=shopping_dates_start_row + max_weeks - 1, min_col=1, max_col=2):
#     for cell in row:
#         if cell.row % 2 == 0:
#             cell.fill = even_fill

# # Apply conditional formatting for the "Status" column
# red_fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")
# orange_fill = PatternFill(start_color="FFD966", end_color="FFD966", fill_type="solid")
# green_fill = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")

# red_rule = Rule(type="containsText", operator="containsText", text="MISSED", dxf=DifferentialStyle(fill=red_fill))
# orange_rule = Rule(type="containsText", operator="containsText", text="TODO", dxf=DifferentialStyle(fill=orange_fill))
# green_rule = Rule(type="containsText", operator="containsText", text="DONE", dxf=DifferentialStyle(fill=green_fill))

# ws_overview.conditional_formatting.add(f'B{shopping_dates_start_row}:B{shopping_dates_start_row + max_weeks - 1}', red_rule)
# ws_overview.conditional_formatting.add(f'B{shopping_dates_start_row}:B{shopping_dates_start_row + max_weeks - 1}', orange_rule)
# ws_overview.conditional_formatting.add(f'B{shopping_dates_start_row}:B{shopping_dates_start_row + max_weeks - 1}', green_rule)

# # Add data validation for shopping status
# dv_status = DataValidation(type="list", formula1='"TODO,DONE,MISSED"', allow_blank=True)
# ws_overview.add_data_validation(dv_status)
# for row in range(max_weeks):
#     cell_number = f'B{shopping_dates_start_row + row}'
#     cell = ws_overview[cell_number]
#     dv_status.add(cell)

# # Add formula to calculate the next occurrence of the preferred shopping day
# ws_overview[f'A{shopping_dates_start_row}'] = f'=A{start_row} + MOD(7 + MATCH(B{start_row}, {"Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"}, 0) - WEEKDAY(A{start_row}, 2), 7)'

# # Add formula to calculate subsequent shopping dates
# for i in range(1, max_weeks):
#     ws_overview[f'A{shopping_dates_start_row + i}'] = f'=A{shopping_dates_start_row + i - 1} + 7'

# Save the workbook
preferred_day = "Monday"
start_date = datetime.now()

create_weekly_sheets(wb, df, start_date, preferred_day)

# ws_overview.append([""])
# score_cell = ws_overview.cell(row=ws_overview.max_row + 1, column=1)
# score_cell.value = "Current Score:"

# # Set the formula for calculating the score dynamically
# score_formula = f'=IFERROR(COUNTIF(B{len(df) + 8}:B{len(df) + 8 + len(shopping_dates) - 1}, "DONE") / (COUNTIF(B{len(df) + 8}:B{len(df) + 8 + len(shopping_dates) - 1}, "DONE") + COUNTIF(B{len(df) + 8}:B{len(df) + 8 + len(shopping_dates) - 1}, "MISSED")), 0)'
# score_value_cell = ws_overview.cell(row=ws_overview.max_row, column=2)
# score_value_cell.value = score_formula

# # Optionally format the score as a percentage
# score_value_cell.number_format = '0.00%'

# Save the workbook
wb.save(output_path)

In [55]:
# SHOPPING DATES TABLE
################################################################

# # Add shopping status table
# ws_overview.append([""])
# ws_overview.append(["Shopping Date", "Status"])
# current_date = datetime.now()
# preferred_day = "Monday"  # Placeholder, can be replaced with user input
# shopping_dates = [current_date + timedelta(days=(i * 7)) for i in range(1, df['Shopping Period (weeks)'].max() + 1)]

# for date in shopping_dates:
#     ws_overview.append([date.strftime("%Y-%m-%d")])

# # Add data validation for shopping status
# dv_status = DataValidation(type="list", formula1='"TODO,DONE,MISSED"', allow_blank=True)
# ws_overview.add_data_validation(dv_status)
# dv_status.error = 'Your entry is not in the list'
# dv_status.errorTitle = 'Invalid Entry'
# dv_status.prompt = 'Please select from the list'
# dv_status.promptTitle = 'List Selection'

# for row in range(len(shopping_dates)):
#     cell_number = f'B{len(df) + 8 + row}'
#     cell = ws_overview[cell_number]
#     cell.value = "TODO"
#     dv_status.add(cell)


# # Set start date and preferred shopping day
# start_date = datetime.now()
# preferred_day = 'Monday'  # Change this based on user input

# # Create weekly sheets
# create_weekly_sheets(wb, df, start_date, preferred_day)

# ws_overview.append([""])
# score_cell = ws_overview.cell(row=ws_overview.max_row + 1, column=1)
# score_cell.value = "Current Score:"

# # Set the formula for calculating the score dynamically
# score_formula = f'=IFERROR(COUNTIF(B{len(df) + 8}:B{len(df) + 8 + len(shopping_dates) - 1}, "DONE") / (COUNTIF(B{len(df) + 8}:B{len(df) + 8 + len(shopping_dates) - 1}, "DONE") + COUNTIF(B{len(df) + 8}:B{len(df) + 8 + len(shopping_dates) - 1}, "MISSED")), 0)'
# score_value_cell = ws_overview.cell(row=ws_overview.max_row, column=2)
# score_value_cell.value = score_formula

# # Optionally format the score as a percentage
# score_value_cell.number_format = '0.00%'

# # Save the workbook
# wb.save(output_path)


In [28]:
# def calculate_fullness_factor(row):
#     CAL = max(30, row['Energy.Energy [KCAL]'])  # Minimum 30
#     PR = min(30, row['Macronutrient.Protein [G]'])  # Maximum 30
#     DF = min(12, row['Macronutrient.Fiber [G]'])  # Maximum 12
#     TF = min(50, row['Macronutrient.Total Fat [G]'])  # Maximum 50
    
#     # Fullness Factor formula
#     FF = max(0.0, min(5.0, 41.7 / (CAL**0.7) + 0.05 * PR + 6.17e-4 * (DF**3) - 7.25e-6 * (TF**3) + 0.617))
#     return FF

# import pandas as pd

# data_path = "../data/processed/merged_rewe_fdc_data.csv"
# df = pd.read_csv(data_path)
# df["Non Nutrient Data.Fullness Factor"] = df.apply(calculate_fullness_factor, axis=1)
# df.to_csv(data_path, index=False)